# Metadata view를 통한 결론

* 탐색한 파일

    * <code>ptbxl_database.csv</code>

    * <code>scp_statements.csv</code>
* * *
# How to get Label
1. <code>ptbxl_database.csv</code>의 <code>scp_codes</code>의 문자열(딕셔너리)을 <code>json.loads()</code>를 통해 가져올 것

    * 이때, Decode할 때 문법적으로 고쳐야하는 거 있으니 아래 참고!

2. 이제 이 likelihood를 어떻게 활용해서 labeling으로 만들 것인가..

    * likelihood가 100이고, 나머지 key 값에 대해서는 0인 순수한 sample에 대해서 label로 매길 것
    
    * 그럼 애매한 데이터는 안 넣은 것이냐? 안 넣은 것이라면, 그 이유는?

        * 애매한 데이터는 말 그대로 특정 label로 labeling이 애매하기 때문이다..

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
ptbxl_db = pd.read_csv(r"data\PTB-XL\ptbxl_database.csv")

In [3]:
scp = pd.read_csv(r"data\PTB-XL\scp_statements.csv")

In [4]:
ptbxl_db.head()

,ecg_id,patient_id,age,sex,height,weight,nurse,site,device,recording_date,...,validated_by_human,baseline_drift,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr
0,1,15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,...,True,NaN,", I-V1,",NaN,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr
1,2,13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,...,True,NaN,NaN,NaN,NaN,NaN,NaN,2,records100/00000/00002_lr,records500/00000/00002_hr
2,3,20372.0,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1984-11-15 12:49:10,...,True,NaN,NaN,NaN,NaN,NaN,NaN,5,records100/00000/00003_lr,records500/00000/00003_hr
3,4,17014.0,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1984-11-15 13:44:57,...,True,", II,III,AVF",NaN,NaN,NaN,NaN,NaN,3,records100/00000/00004_lr,records500/00000/00004_hr
4,5,17448.0,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1984-11-17 10:43:15,...,True,", III,AVR,AVF",NaN,NaN,NaN,NaN,NaN,4,records100/00000/00005_lr,records500/00000/00005_hr


In [5]:
scp.head()

,Unnamed: 0,description,diagnostic,form,rhythm,diagnostic_class,diagnostic_subclass,Statement Category,SCP-ECG Statement Description,AHA code,aECG REFID,CDISC Code,DICOM Code
0,NDT,non-diagnostic T abnormalities,1.0,1.0,NaN,STTC,STTC,other ST-T descriptive statements,non-diagnostic T abnormalities,NaN,NaN,NaN,NaN
1,NST_,non-specific ST changes,1.0,1.0,NaN,STTC,NST_,Basic roots for coding ST-T changes and abnorm...,non-specific ST changes,145.0,MDC_ECG_RHY_STHILOST,NaN,NaN
2,DIG,digitalis-effect,1.0,1.0,NaN,STTC,STTC,other ST-T descriptive statements,suggests digitalis-effect,205.0,NaN,NaN,NaN
3,LNGQT,long QT-interval,1.0,1.0,NaN,STTC,STTC,other ST-T descriptive statements,long QT-interval,148.0,NaN,NaN,NaN
4,NORM,normal ECG,1.0,NaN,NaN,NORM,NORM,Normal/abnormal,normal ECG,1.0,NaN,NaN,F-000B7


In [6]:
ptbxl_db.columns

Index(['ecg_id', 'patient_id', 'age', 'sex', 'height', 'weight', 'nurse',
       'site', 'device', 'recording_date', 'report', 'scp_codes', 'heart_axis',
       'infarction_stadium1', 'infarction_stadium2', 'validated_by',
       'second_opinion', 'initial_autogenerated_report', 'validated_by_human',
       'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems',
       'extra_beats', 'pacemaker', 'strat_fold', 'filename_lr', 'filename_hr'],
      dtype='object')

In [7]:
scp.columns

Index(['Unnamed: 0', 'description', 'diagnostic', 'form', 'rhythm',
       'diagnostic_class', 'diagnostic_subclass', 'Statement Category',
       'SCP-ECG Statement Description', 'AHA code', 'aECG REFID', 'CDISC Code',
       'DICOM Code'],
      dtype='object')

In [8]:
print(ptbxl_db['scp_codes'][0])

{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}


In [9]:
print(ptbxl_db['scp_codes'][0].replace("\'", "\""))

{"NORM": 100.0, "LVOLT": 0.0, "SR": 0.0}


In [10]:
len(ptbxl_db)

21799

In [16]:
# 중요! : 이렇게 바꿔야 json 모듈에서 decode한다.

cnt_dict = {
    'NORM': 0,
    'NDT': 0,
    'NST_': 0,
    'DIG': 0,
    'LNGQT': 0
}

def check_normal(a):
    # 완전히 순수한 정상 데이터
    ret = False
    for k, v in a.items():
        if (k == 'NORM' and v == 100):
            ret = True
        elif (k != 'NORM' and v == 0): 
            continue
        else: 
            return False
    return ret

def check_abnormal(a, label: str):
    # label에 대한 Abnormality는 100이지만, 다른 것에 대한 likelihood는 100만 아니면 된다.
    ret = False
    for k, v in a.items():
        if (k == label and v == 100):
            ret = True
        elif (k != label and v == 0): 
            continue
        else: 
            return False
    return ret

for i in range(len(ptbxl_db)):
    a = json.loads(ptbxl_db['scp_codes'][i].replace("\'", "\""))
    
    if check_normal(a):
        cnt_dict['NORM'] += 1
        
    for label in cnt_dict.keys():
        if label == 'NORM':
            continue
        if check_abnormal(a, label):
            cnt_dict[label] += 1
    
abnormal_sum = 0

print(f"[Total Normal Data]: {cnt_dict['NORM']}")

for k, v in cnt_dict.items():
    if k == 'NORM':
        continue
    print(f"{k}: {v}")
    abnormal_sum += v
print(f"[Total Abnormal Data]: {abnormal_sum}")

[Total Normal Data]: 7017
NDT: 1182
NST_: 374
DIG: 12
LNGQT: 40
[Total Abnormal Data]: 1608
